In [ ]:
import os, json, torch, time, igl, sys
sys.path.append("../src/")
import polyfempy as pf
import numpy as np
from lbfgs import LBFGS
from stressNorm import StressNormLoss
from volume import VolumeLoss
from smooth import BoundarySmooth
from slim import smooth_internal_nodes, mesh_quality

torch.set_default_dtype(torch.float64)

In [ ]:
# Differentiable simulator that computes shape derivatives
class Simulate(torch.autograd.Function):

    @staticmethod
    def forward(ctx, solver, vertices):
        # Update solver setup
        solver.mesh().set_vertices(vertices)
        # Enable caching intermediate variables in the simulation, which will be used for solve_adjoint
        solver.set_cache_level(pf.CacheLevel.Derivatives)
        # Run simulation
        solver.solve()
        # Collect transient simulation solutions
        sol = torch.tensor(solver.get_solutions())
        # Cache solver for backward gradient propagation
        ctx.solver = solver
        return sol

    @staticmethod
    @torch.autograd.function.once_differentiable
    def backward(ctx, grad_output):
        # solve_adjoint only needs to be called once per solver, independent of number of types of optimization variables
        ctx.solver.solve_adjoint(grad_output)
        # Compute shape derivatives
        return None, torch.tensor(pf.shape_derivative(ctx.solver))

In [ ]:
# Fix the geometry at Dirichlet & Neumann boundary
def compute_fix_vertex_mask(mesh):
    mask = np.array([not mesh.is_boundary_vertex(i) for i in range(mesh.n_vertices())], dtype=bool)
    for fid in range(mesh.n_boundary_elements()):
        if mesh.get_boundary_id(fid) in [10, 11]:
            for lv in range(2):
                mask[mesh.boundary_element_vertex(fid, lv)] = True
    return mask

In [ ]:
root = os.getcwd()
with open(root + "/run.json", "r") as f:
    config = json.load(f)
    config["root_path"] = root + "/run.json"

log_level = 3 # warning

# Simulation
solver = pf.Solver()
solver.set_settings(json.dumps(config), True)
solver.set_log_level(log_level)
solver.load_mesh_from_settings()

mesh = solver.mesh()
v = mesh.vertices()
vertices = torch.tensor(solver.mesh().vertices(), requires_grad=True)
fix_vertex_mask = compute_fix_vertex_mask(mesh)

with torch.no_grad():
    initial_volume = float(VolumeLoss(vertices, mesh.elements()))
    print("Initial volume", initial_volume)

def loss(vertices):
    areas = igl.doublearea(vertices.detach().numpy(), solver.mesh().elements())
    if np.min(areas) <= 0:
        return float("nan")

    try:
        solutions = Simulate.apply(solver, vertices)
    except:
        return float("nan")

    return pow(StressNormLoss.apply(solver, solutions, vertices, 8), 1/8) + \
            1e5 * torch.maximum(VolumeLoss(vertices, mesh.elements()) - initial_volume, torch.tensor(0.)) ** 2 + \
            1e4 * BoundarySmooth.apply(solver, vertices, [0])

In [ ]:
def verify_grad(vertices):
    param = vertices.clone().detach().requires_grad_(True)
    theta = torch.randn_like(param)
    theta[fix_vertex_mask, :] = 0
    l = loss(param)
    l.backward()
    grad = param.grad
    t = 1e-7
    with torch.no_grad():
        analytic = torch.tensordot(grad, theta)
        f1 = loss(param + theta * t)
        f2 = loss(param - theta * t)
        fd = (f1 - f2) / (2 * t)
        print(f'grad {analytic}, fd {fd} {(f1 - l) / t} {(l - f2) / t}, relative err {abs(analytic - fd) / abs(analytic):.3e}')
        print(f'f(x+dx)={f1}, f(x)={l.detach()}, f(x-dx)={f2}')
        assert(abs(analytic - fd) <= 1e-5 * abs(analytic))

In [ ]:
def save_to_file(x, iter, out_dir):
    sys.stdout.flush()
    outpath = os.path.join(out_dir, "iter_" + str(iter) + ".vtu")
    print("Save to", outpath)
    solver.export_vtu(outpath, solver.get_solutions())

In [ ]:
# The bridge example does not need remesh
for remesh_iter in range(1):
    mesh = solver.mesh()
    faces = mesh.elements()
    
    boundary_vertex_mask = np.zeros((mesh.n_vertices()), dtype=bool)
    boundary_vertex_mask[igl.boundary_facets(faces)] = True
    boundary_vertex_indices = np.argwhere(boundary_vertex_mask).flatten()

    fix_vertex_mask = compute_fix_vertex_mask(mesh)
    flexible_vertex_mask = np.logical_not(fix_vertex_mask)

    full_param = torch.tensor(mesh.vertices())
    param = full_param[flexible_vertex_mask, :].clone().requires_grad_(True)
    optimizer = LBFGS([param], history_size=6, max_iter=1, lr=1, line_search_fn='back_tracking')

    # optimization configuration
    out_dir = os.path.join(os.getcwd(), "opt_" + str(remesh_iter))
    if os.path.exists(out_dir):
        os.system("rm -r " + out_dir)
    os.mkdir(out_dir)

    # run simulation and compute loss
    def closure():
        optimizer.zero_grad()
        full_param_tmp = full_param.clone()
        full_param_tmp[flexible_vertex_mask, :] = param
        l = loss(full_param_tmp)
        if isinstance(l, torch.Tensor):
            l.backward()
        return l

    start_time = time.time()
    need_remesh = False
    for iter in range(100):
        last_loss = closure()
        print(f'Step {iter}: energy {last_loss:.4e}, grad norm {torch.linalg.norm(param.grad):.4e}, total time {time.time() - start_time:.2f} sec')
        save_to_file(param, iter, out_dir)
        prev_vertices = mesh.vertices().copy()

        # Let optimizer take a step
        optimizer.step(closure)
       
        # Once optimizer takes a step on the boundary vertices, move internal vertices using igl.SLIM
        with torch.no_grad():
            full_param[flexible_vertex_mask, :] = param
            full_param = torch.tensor(smooth_internal_nodes(prev_vertices, faces, boundary_vertex_indices, full_param[boundary_vertex_indices, :].detach().numpy()))
        
        # Remesh if the mesh quality is bad and cannot be resolved with SLIM
        new_vertices = full_param.detach().numpy()
        quality = mesh_quality(new_vertices, faces)
        print("mesh quality", quality)
        if quality > 100:
            need_remesh = True
            new_vertices = np.hstack((new_vertices, np.zeros((new_vertices.shape[0], 1))))
            igl.write_triangle_mesh("before_remesh_" + str(remesh_iter) + ".obj", new_vertices, faces)
            break
    
    if not need_remesh:
        break
    
    print("Bad mesh quality, remesh...")
    os.system("python ../src/remesh.py " + \
              "before_remesh_" + str(remesh_iter) + ".obj" + " " + \
              os.path.join(root, "after_remesh_" + str(remesh_iter) + ".msh") + " > remesh_" + str(remesh_iter) + ".log")

    print("Remesh finished!")
    solver = pf.Solver()
    config["geometry"][0]["mesh"] = "after_remesh_" + str(remesh_iter) + ".msh"
    solver.set_settings(json.dumps(config), True)
    solver.set_log_level(log_level)
    solver.load_mesh_from_settings()

print("Optimization finished!")